# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.

## Descripción de los datos

Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la columna `num_orders`.

## Preparación

#### Definir el tipo de proyecto

De acuerdo a las caracteristicas mencionadas del proycto, nos encontramos ante un objetivo de predicción a través de modelos de regresión.

#### Importación de librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from pmdarima.arima import auto_arima
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics



#### Importación de dataframe

In [2]:
# df = pd.read_csv('/datasets/taxi.csv',parse_dates=   ,index_col=)
df = pd.read_csv('D:/Tripleten/datasets/taxi.csv', index_col='datetime', parse_dates=[0] ) #
df.sort_index(inplace=True)

#### Visión general del df

In [3]:
print(df.head(), end='\n\n')
df.info()

                     num_orders
datetime                       
2018-03-01 00:00:00           9
2018-03-01 00:10:00          14
2018-03-01 00:20:00          28
2018-03-01 00:30:00          20
2018-03-01 00:40:00          32

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


#### Revision de duplicados

In [4]:
df.index.duplicated().sum()

0

#### Revisión de valores nulos

De acuerdo con df.info() no existen valores nulos

#### Coherencia de datos

Los datos mantienen una tendencia de crecimiento, por el momento muestran coherencia.

In [5]:
df.resample('W').sum().head(10)

,num_orders
datetime,
2018-03-04,5301
2018-03-11,9411
2018-03-18,9785
2018-03-25,9767
2018-04-01,9752
2018-04-08,9377
2018-04-15,10352
2018-04-22,11224
2018-04-29,12041


- [X] Identificar tipo de proyecto
- [X] Nombre de columna correcta
- [X] Clasificaciones de tipo de dato correctas
- [X] Dataframe sin duplicados
- [X] Excluir valores nulos
- [X] Verificar la coherencia de los datos

Conclusiones del proceso de preparación.

El conjunto de datos cargado mantiene una limpieza en sus datos (nombre de columna correctamente nombrados, coherencia de datos, inexistencia de duplicados y valores nulos ). Los datos cargados establecen un periodo desde el periodo 2018-03-01 hasta 2018-08-31.

## Análisis

In [6]:
# df.resample('YE').sum()
df_monthly = df.resample('ME').sum()
df_weekly = df.resample('W').sum()
df_daily = df.resample('D').sum()
df_hourly = df.resample('h').sum()

In [7]:
# titles = ['Gráfico mensual', 'Gráfico semanal', 'Gráfico diario', 'Gráfico por hora']
# interval = [1,4,24,480]

# databases = [df_monthly,df_weekly,df_daily, df_hourly]
# fig, axis = plt.subplots(4,1, figsize=[10,5])

# for i, database in enumerate(databases):
#     axis[i].plot(database.index, database.values, '--o')
#     axis[i].set_title(titles[i])
#     axis[i].set_xticks(database.index)
#     axis[i].set_xticks(database.index[::interval[i]])

# plt.tight_layout()
# plt.show()

In [8]:
decomposed = seasonal_decompose(df_hourly, model='additive', period =7)
# decomposed.plot()
# plt.title('Orders Trend')
# plt.show()

## Formación

Definiendo función para crear mas columnas de información

In [9]:
def make_features(df, max_lag, rolling_mean_size):
    '''Esta función tomará una serie temporal y creara columnas para aumentar su posibilidad de predicción
     en modelos de machine learning, generará columnas de año, mes, día, día de la semana y generará desfaces en los valores definidos 
    por la entrada, también generará una columna con el promedio móvil'''

    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek

    for lag in range(1, max_lag +1):
        df[f'lag_{lag}'] = df['num_orders'].shift(lag)

    # df['rolling_mean'] = df['num_orders'].rolling(rolling_mean_size).mean()
    df['rolling_mean'] = df['num_orders'].shift().rolling(rolling_mean_size).mean()

make_features(df_hourly,3,3)

df_hourly.dropna(inplace=True)

In [10]:

y= df_hourly['num_orders']
X= df_hourly.drop(columns='num_orders')

X_train, X_test,y_train, y_test = train_test_split(X,y,shuffle=False, test_size=1/10) #10% del conjuntos de datos inicial

## Prueba

Modelo Linear Regression

In [11]:
model = LinearRegression()
model.fit(X_train, y_train)
test_y_predict = model.predict(X_test)
test_rmse_score = root_mean_squared_error(y_test,test_y_predict)

y_predict = model.predict(X)
rmse_score = root_mean_squared_error(y,y_predict)

print(f'Test Score:{test_rmse_score}')
print(f'DF_Score:{rmse_score}')

Test Score:52.45420941511783
DF_Score:33.44602050649104


In [23]:
lenght = len(test_y_predict)
print(f'Los demás ejercicios los  haremos con predicción de longitud {lenght}')

Los demás ejercicios los  haremos con predicción de longitud 442


Modelo Auto Arima

In [25]:
df_arima = df_hourly['num_orders']
train, test = train_test_split(df_arima, shuffle=False, test_size=1/10)

In [27]:
model = auto_arima(train, seasonal=True, m=len(test) )
summary = model.summary()
predict = model.predict(n_periods=len(test))

rmse_score = root_mean_squared_error(test, predict)
print(f'DF_Score:{rmse_score}')

Modelo Prophet (Meta)

In [ ]:
# df_prophet = pd.DataFrame(data={"ds":df_hourly.index,"y":df_hourly['num_orders']})
# df_prophet = df_prophet.reset_index(drop=True)

# model = Prophet()
# model.fit(df_prophet)

# #Creando mas periodos
# future = model.make_future_dataframe(periods=24, freq='H') 

# # Realizar las predicciones
# forecast = model.predict(future)

# # Definir el número de periodos hacia adelante para la validación cruzada
# horizon = '12 days'

# # Definir el tamaño de cada corte en la validación cruzada
# initial = '5 days'  # Por ejemplo, 120 días de datos iniciales

# # Definir el período de ventana entre cada corte en la validación cruzada
# period = '24 hours'  # Por ejemplo, una ventana de 24 horas entre cada corte

# # Realizar la validación cruzada
# df_cv = cross_validation(model, horizon=horizon, initial=initial, period=period)

15:11:25 - cmdstanpy - INFO - Chain [1] start processing
15:11:25 - cmdstanpy - INFO - Chain [1] done processing


C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\prophet\forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
Seasonality has period of 7 days which is larger than initial window. Consider increasing initial.
  0%|          | 0/167 [00:00<?, ?it/s]15:11:27 - cmdstanpy - INFO - Chain [1] start processing
15:11:27 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 1/167 [00:00<00:47,  3.51it/s]15:11:27 - cmdstanpy - INFO - Chain [1] start processing
15:11:27 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 2/167 [00:00<00:48,  3.40it/s]15:11:28 - cmdstanpy - INFO - Chain [1] start processing
15:11:28 - cmdstanpy - INFO - Chain [1] done processing
  2%|▏         | 3/167 [00:00<00:48,  3.37it/s]15:11:28 - cmdstanpy - INFO - Chain [1] start processing
15:11:28 - cmdstanpy - INFO - Chain

In [ ]:
# df_prophet = pd.DataFrame(data={"ds":df_hourly.index,"y":df_hourly['num_orders']})
# df_prophet = df_prophet.reset_index(drop=True)

# model = Prophet()
# model.fit(df_prophet)

# #Creando mas periodos
# future = model.make_future_dataframe(periods=24, freq='H') 

# # Realizar las predicciones
# forecast = model.predict(future)

# # Definir el número de periodos hacia adelante para la validación cruzada
# horizon = '12 days'

# # Definir el tamaño de cada corte en la validación cruzada
# initial = '5 days'  # Por ejemplo, 120 días de datos iniciales

# # Definir el período de ventana entre cada corte en la validación cruzada
# period = '24 hours'  # Por ejemplo, una ventana de 24 horas entre cada corte

# # Realizar la validación cruzada
# df_cv = cross_validation(model, horizon=horizon, initial=initial, period=period)

15:11:25 - cmdstanpy - INFO - Chain [1] start processing
15:11:25 - cmdstanpy - INFO - Chain [1] done processing


C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\prophet\forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
Seasonality has period of 7 days which is larger than initial window. Consider increasing initial.
  0%|          | 0/167 [00:00<?, ?it/s]15:11:27 - cmdstanpy - INFO - Chain [1] start processing
15:11:27 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 1/167 [00:00<00:47,  3.51it/s]15:11:27 - cmdstanpy - INFO - Chain [1] start processing
15:11:27 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 2/167 [00:00<00:48,  3.40it/s]15:11:28 - cmdstanpy - INFO - Chain [1] start processing
15:11:28 - cmdstanpy - INFO - Chain [1] done processing
  2%|▏         | 3/167 [00:00<00:48,  3.37it/s]15:11:28 - cmdstanpy - INFO - Chain [1] start processing
15:11:28 - cmdstanpy - INFO - Chain

In [ ]:
# df_p = performance_metrics(df_cv)
# df_p

,horizon,mse,rmse,mae,mdape,smape,coverage
0,1 days 05:00:00,830.801908,28.823635,20.979693,0.214494,0.271116,0.733230
1,1 days 06:00:00,781.161310,27.949263,20.332001,0.216285,0.275815,0.747447
2,1 days 07:00:00,744.318407,27.282199,19.856921,0.220591,0.287958,0.758669
3,1 days 08:00:00,693.999622,26.343873,19.496465,0.225308,0.296710,0.765435
4,1 days 09:00:00,669.082453,25.866628,19.125994,0.223226,0.294526,0.772016
...,...,...,...,...,...,...,...
255,11 days 20:00:00,866.325540,29.433409,21.712113,0.217519,0.277877,0.705212
256,11 days 21:00:00,856.586191,29.267494,21.546989,0.216475,0.276861,0.710807
257,11 days 22:00:00,837.743051,28.943791,21.270372,0.215517,0.274789,0.718108
258,11 days 23:00:00,841.466875,29.008048,21.415609,0.215687,0.274497,0.712625


# Lista de revisión

- [x]  Jupyter Notebook está abierto.
- [ ]  El código no tiene errores
- [ ]  Las celdas con el código han sido colocadas en el orden de ejecución.
- [ ]  Los datos han sido descargados y preparados.
- [ ]  Se ha realizado el paso 2: los datos han sido analizados
- [ ]  Se entrenó el modelo y se seleccionaron los hiperparámetros
- [ ]  Se han evaluado los modelos. Se expuso una conclusión
- [ ]  La *RECM* para el conjunto de prueba no es más de 48